In [3]:
import tensorflow as tf
import pickle
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers

In [10]:
import pandas as pd

# charge prediction

In [11]:
df_kb = pd.read_csv('../data/massIVE-KB/df_kb_noptm.csv')

In [12]:
print(df_kb.shape)
df_kb.head(2)

(1163969, 6)


,CHARGE,PEPTIDE,PROTEINS,SPECTRAL_CNT,MZ,INTENSITY
0,2,HGYIPGK,sp|Q9ULV4|COR1C_HUMAN,20,110.07217407226562;110.0797119140625;111.07563...,109124.765625;3682.905029296875;3920.083740234...
1,2,KEGLSFNK,sp|Q6UVY6|MOXD1_HUMAN,4,101.10691833496094;102.05462646484375;110.0711...,561.285400390625;362.3951110839844;388.3195495...


In [13]:
pep2charge = dict()
for p, c in df_kb[['PEPTIDE', 'CHARGE']].values:
    if p not in pep2charge:
        pep2charge[p] = [0, 0, 0, 0, 0, 0]
    charge_idx = c-1
    pep2charge[p][charge_idx] += 1

In [ ]:
pep_data = 

In [ ]:
def label2onehot(pep_list):
    pep_data = [[[1 if aa == a else 0 for a in 'ARNDCQEGHILKMFPSTWYV']
             for aa in seq] + [[0]*20]*(30-len(seq))
             for seq in pep]
    return np.array(pep_data)

In [24]:
def reshapeOneHot(X):
    X = np.dstack(X)
    X = np.swapaxes(X, 1, 2)
    X = np.swapaxes(X, 0, 1)
    return X

def one_hot_encode_peptide(peptide, MAX_LENGTH = 30):
    if len(peptide) > MAX_LENGTH:
        print('Peptide length is larger than maximal length of ', str(MAX_LENGTH))
        return None
    else:
        AA_vocabulary = 'KRPTNAQVSGILCMJHFYWEDBXOU'#B: acetyl; O: Carbamyl; J: oxidized Met; X:pyro_glu
        no_not_used_aas = 1#U: not used

        one_hot_peptide = np.zeros((len(peptide), len(AA_vocabulary) - no_not_used_aas))

        for j in range(0, len(peptide)):
            try:
                aa = peptide[j]
                one_hot_peptide[j, AA_vocabulary.index(aa)] = 1
            except:
                pass
        
        no_front_paddings = int((MAX_LENGTH - len(peptide))/2)
        peptide_front_paddings = np.zeros((no_front_paddings, one_hot_peptide.shape[1]))

        no_back_paddings = MAX_LENGTH - len(peptide) - no_front_paddings
        peptide_back_paddings = np.zeros((no_back_paddings, one_hot_peptide.shape[1]))

        full_one_hot_peptide = np.vstack((peptide_front_paddings, one_hot_peptide, peptide_back_paddings))

        return peptide, full_one_hot_peptide

seq = '(Acetyl)-AC(Carbamidomethyl)DEM(Oxidation)NN(Deamidated)QQ(Deamidated)K'
ohc_seq = one_hot_encode_peptide(seq)

In [35]:
from tensorflow.keras import models
from tensorflow.keras import layers

network = models.Sequential()
network.add(layers.Masking(mask_value=0., input_shape=(None, training_X.shape[2])))
network.add(layers.Bidirectional(layers.LSTM(256,
			 		     dropout=0.1,
					     recurrent_dropout=0.5,  
					     return_sequences=True)))
network.add(layers.Bidirectional(layers.LSTM(256,
			 		     dropout=0.1,
					     recurrent_dropout=0.5)))
network.add(layers.Dense(256, activation='tanh'))
network.add(layers.Dense(5, activation='softmax'))
print(network.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 23)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, None, 512)         573440    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1285      
Total params: 2,280,965
Trainable params: 2,280,965
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
network.compile(optimizer='adam', loss='mean_squared_error')
history = network.fit(training_X, 
		      training_y, 
		      epochs=150, 
		      batch_size=1024,
		      validation_split=0.2)

cost = network.evaluate(testing_X, testing_y, batch_size=1024)
print('test cost:', cost)